In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import catboost as cb
from catboost import CatBoostClassifier, Pool, EFeaturesSelectionAlgorithm
import time

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, roc_auc_score, average_precision_score, precision_score, recall_score, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split

from cc5_preprocessor import Preprocessor

import joblib

np.random.seed(42)

In [2]:
df = pd.read_csv('data/Training_TriGuard.csv')
df = df.dropna(subset=['subrogation'])

In [3]:
pre = Preprocessor(smoothing_factor=5, mode = 'catboost')

In [4]:
X = df.drop(columns=["subrogation"]).copy()
y = df["subrogation"].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=0)

In [5]:
y_train.value_counts(normalize=True)

subrogation
0.0    0.77141
1.0    0.22859
Name: proportion, dtype: float64

In [6]:
y_test.value_counts(normalize=True)

subrogation
0.0    0.771296
1.0    0.228704
Name: proportion, dtype: float64

In [7]:
pre.fit(X_train, y_train)

X_train_proc = pre.transform(X_train)
X_test_proc = pre.transform(X_test)

X_test_proc = X_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)

Fitting Preprocessor in 'catboost' mode...
CatBoost mode: Skipping target encoding learning.
Learning statistical parameters for Z-scoring...
Fit complete.
Transforming data in 'catboost' mode...
Applying learned statistical transforms (Z-scores)...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
Transforming data in 'catboost' mode...
Applying learned statistical transforms (Z-scores)...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.


In [8]:
print("Saving preprocessor and training columns...")

# Save the 'pre' object
joblib.dump(pre, 'cc3_preprocessor.pkl')

# Save the exact column order and names
joblib.dump(X_train_proc.columns, 'training_columns.pkl')

print("Done.")

Saving preprocessor and training columns...
Done.


## CatBoost with Optuna Tuning

In [9]:
import optuna
from optuna.integration import CatBoostPruningCallback

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
CAT_FEATURES = pre.cat_for_encoding_
print(CAT_FEATURES)

['accident_site', 'accident_type', 'channel', 'vehicle_category', 'vehicle_color', 'living_status', 'claim_day_of_week', 'gender', 'in_network_bodyshop', 'season']


In [11]:
def objective(trial: optuna.trial.Trial) -> float:

    params = {
        'iterations': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'depth': trial.suggest_int('depth', 3, 10),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.1),
        'random_strength': trial.suggest_float('random_strength', 1e-8, 1.0, log=True), 
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0),
        
        'eval_metric': 'F1',
        'task_type': 'CPU',
        'verbose': False,
        'early_stopping_rounds': 100,
        'random_state': 123
    }

    params['eval_metric'] = 'Logloss'
    
    model = CatBoostClassifier(**params)
    
    model.fit(
        X_train_proc, y_train,
        eval_set=(X_test_proc, y_test),
        cat_features=CAT_FEATURES,
        verbose=False
    )

    y_preds = model.predict(X_test_proc)
    
    manual_f1_score = f1_score(y_test, y_preds, pos_label=1)
    
    return manual_f1_score

In [13]:
print("\n2. Starting Optuna study...")

study = optuna.create_study(
    direction='maximize',
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)
)

study.optimize(
    objective, 
    n_trials=100, # Number of trials to run
    show_progress_bar=True
)

print("\n" + "="*50)
print("Optuna study finished.")
print(f"Number of finished trials: {len(study.trials)}")

print("\nBest trial:")
best_trial = study.best_trial
    
print(f"  Value (Max F1 Score): {best_trial.value:.4f}") # <-- CHANGED comment
    
print("  Best Hyperparameters:")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

[I 2025-11-12 12:16:40,918] A new study created in memory with name: no-name-4ee938df-9a18-4c7b-93fa-318c908af48d



2. Starting Optuna study...


Best trial: 0. Best value: 0.589641:   1%|          | 1/100 [00:02<04:32,  2.75s/it]

[I 2025-11-12 12:16:43,673] Trial 0 finished with value: 0.5896414342629482 and parameters: {'learning_rate': 0.03945611717792594, 'depth': 6, 'l2_leaf_reg': 0.005339045005626586, 'subsample': 0.8, 'random_strength': 1.0685974307072054e-05, 'bagging_temperature': 0.7661345098981627, 'border_count': 207, 'scale_pos_weight': 2.0964408316048657}. Best is trial 0 with value: 0.5896414342629482.


Best trial: 0. Best value: 0.589641:   2%|▏         | 2/100 [00:04<03:09,  1.93s/it]

...

[I 2025-11-12 12:25:00,064] Trial 97 finished with value: 0.597544338335607 and parameters: {'learning_rate': 0.019446071509540686, 'depth': 6, 'l2_leaf_reg': 9.981295445259166, 'subsample': 0.5, 'random_strength': 0.003204928638265681, 'bagging_temperature': 0.5025999986312374, 'border_count': 188, 'scale_pos_weight': 2.4448368846582262}. Best is trial 36 with value: 0.604399323181049.


Best trial: 36. Best value: 0.604399:  99%|█████████▉| 99/100 [08:24<00:04,  4.46s/it]

[I 2025-11-12 12:25:04,950] Trial 98 finished with value: 0.5853840417598807 and parameters: {'learning_rate': 0.019358512501146077, 'depth': 6, 'l2_leaf_reg': 0.015006796120480332, 'subsample': 0.6, 'random_strength': 0.0027460111625595917, 'bagging_temperature': 0.503484100021838, 'border_count': 191, 'scale_pos_weight': 1.9481725826251683}. Best is trial 36 with value: 0.604399323181049.


Best trial: 36. Best value: 0.604399: 100%|██████████| 100/100 [08:28<00:00,  5.09s/it]

[I 2025-11-12 12:25:09,481] Trial 99 finished with value: 0.5930388219544847 and parameters: {'learning_rate': 0.02390006623923128, 'depth': 5, 'l2_leaf_reg': 9.260999889991405, 'subsample': 0.8, 'random_strength': 0.005724255876258864, 'bagging_temperature': 0.45465569177482723, 'border_count': 201, 'scale_pos_weight': 2.5301278827071667}. Best is trial 36 with value: 0.604399323181049.

Optuna study finished.
Number of finished trials: 100

Best trial:
  Value (Max F1 Score): 0.6044
  Best Hyperparameters:
    learning_rate: 0.010431688073932016
    depth: 6
    l2_leaf_reg: 0.13650220185823378
    subsample: 1.0
    random_strength: 0.003373546298820878
    bagging_temperature: 0.5141077456444894
    border_count: 118
    scale_pos_weight: 2.4566730131089503


In [14]:
best_params_from_optuna = study.best_trial.params.copy()

rfe_params = best_params_from_optuna.copy()
rfe_params.update({
    'iterations': 1000, # Max iterations per step
    'eval_metric': 'Logloss',
    'task_type': 'CPU',
    'early_stopping_rounds': 100,
    'random_state': 123,
    'verbose': False
})

train_pool = Pool(
    data=X_train_proc, 
    label=y_train, 
    cat_features=CAT_FEATURES
)

eval_pool = Pool(
    data=X_test_proc, 
    label=y_test, 
    cat_features=CAT_FEATURES
)

model_for_rfe = CatBoostClassifier(**rfe_params)

print("Starting CatBoost RFE...")
start_time = time.time()

summary = model_for_rfe.select_features(
    train_pool,
    eval_set=eval_pool,
    algorithm=EFeaturesSelectionAlgorithm.RecursiveByLossFunctionChange,
    steps=10, 
    num_features_to_select = 50,
    features_for_select=X_train_proc.columns.tolist(),
    logging_level='Info', # Shows progress
    train_final_model=False
)

end_time = time.time()
print(f"RFE Complete. Took {end_time - start_time:.2f} seconds.")

selected_features = summary['selected_features_names']
eliminated_features = summary['eliminated_features_names']

print(f"\n--- RFE Results ---")
print(f"Total features selected: {len(selected_features)}")
print(f"Total features eliminated: {len(eliminated_features)}")
print("\nEliminated features:")
print(eliminated_features)

Starting CatBoost RFE...
Step #1 out of 10

liab_prct_cubed, bin=36 score 24.98265619
liab_x_multicar, bin=7 score 31.58215735
liab_x_highrisk_site, bin=6 score 33.82888489
witness_binary, bin=0 score 35.48765322
liab_prct_cubed, bin=40 score 36.39629048
liab_prct_cubed, bin=29 score 37.05971309

liab_prct_cubed, bin=36 score 24.58744181
liab_x_multicar, bin=7 score 31.09713309
liab_x_highrisk_site, bin=6 score 33.31774394
liab_x_witness, bin=7 score 34.96616059
liab_inverse_squared, bin=29 score 35.86737153
liab_prct, bin=30 score 36.52739751

liab_inverse_squared, bin=36 score 24.13139734
liab_x_multicar, bin=7 score 30.54758602
liab_x_highrisk_site, bin=6 score 32.74130114
liab_x_witness, bin=7 score 34.37425016
liab_prct, bin=40 score 35.26409402
liab_inverse_squared, bin=42 score 35.91847287

liab_prct_squared, bin=36 score 23.74300728
liab_x_multicar, bin=7 score 30.06973542
liab_x_highrisk_site, bin=6 score 32.24394135
liab_x_witness, bin=7 score 33.85073346
liab_prct_sqrt, bin=

In [15]:
best_params = study.best_trial.params
print(best_params)

final_params = best_params.copy()
final_params.update({
    'iterations': 2000, # Use more iterations for the final model
    'eval_metric': 'Logloss', # Use Logloss for training/stopping
    'task_type': 'CPU',
    'early_stopping_rounds': 100, # Keep early stopping
    'random_state': 123
})

best_model = CatBoostClassifier(**final_params)

best_model.fit(
    X_train_proc, y_train,
    eval_set=(X_test_proc, y_test),
    cat_features=CAT_FEATURES,
    verbose=False
)

print(f"\nFinal Model Score (from best Logloss iteration):")
y_preds_final = best_model.predict(X_test_proc)
final_f1 = f1_score(y_test, y_preds_final, pos_label=1)
print(f"  Manual F1:class=1 Score: {final_f1:.4f}")
        
print("\n  Full Classification Report:")
print(classification_report(y_test, y_preds_final, target_names=['Class 0.0', 'Class 1.0']))

{'learning_rate': 0.010431688073932016, 'depth': 6, 'l2_leaf_reg': 0.13650220185823378, 'subsample': 1.0, 'random_strength': 0.003373546298820878, 'bagging_temperature': 0.5141077456444894, 'border_count': 118, 'scale_pos_weight': 2.4566730131089503}

Final Model Score (from best Logloss iteration):
  Manual F1:class=1 Score: 0.6044

  Full Classification Report:
              precision    recall  f1-score   support

   Class 0.0       0.91      0.80      0.85      4165
   Class 1.0       0.52      0.72      0.60      1235

    accuracy                           0.78      5400
   macro avg       0.71      0.76      0.73      5400
weighted avg       0.82      0.78      0.79      5400



In [16]:
X_train_top_features = X_train_proc[selected_features]
X_test_top_features = X_test_proc[selected_features]

original_cat_features = set(CAT_FEATURES)
top_features_set = set(selected_features)
new_cat_features = list(original_cat_features.intersection(top_features_set))

In [18]:
final_params_rfe = study.best_trial.params.copy()
final_params_rfe.update({
    'iterations': 2000, 
    'eval_metric': 'Logloss',
    'task_type': 'CPU',
    'early_stopping_rounds': 100,
    'random_state': 123
})

new_model = CatBoostClassifier(**final_params_rfe)

new_model.fit(
    X_train_top_features, y_train,
    eval_set=(X_test_top_features, y_test),
    cat_features=new_cat_features,
    verbose=False
)

y_preds_new = new_model.predict(X_test_top_features)
new_f1 = f1_score(y_test, y_preds_new, pos_label=1)

# Get the original F1 score from cell 13 to compare
original_f1_score = f1_score(y_test, best_model.predict(X_test_proc), pos_label=1)

print("\n--- Model Performance Comparison ---")
print(f"Original F1 score (all features): {original_f1_score:.4f}")
print(f"New F1 score ({len(selected_features)} RFE features): {new_f1:.4f}")

print("\nNew Model Classification Report (RFE Features):")
print(classification_report(y_test, y_preds_new, target_names=['Class 0.0', 'Class 1.0']))


--- Model Performance Comparison ---
Original F1 score (all features): 0.6044
New F1 score (50 RFE features): 0.6026

New Model Classification Report (RFE Features):
              precision    recall  f1-score   support

   Class 0.0       0.91      0.80      0.85      4165
   Class 1.0       0.52      0.72      0.60      1235

    accuracy                           0.78      5400
   macro avg       0.71      0.76      0.73      5400
weighted avg       0.82      0.78      0.79      5400



In [19]:
print("Saving best_model...")

# Use the model's F1 score in the name
best_model.save_model("catboost_mod_f1_0.6044.cbm")

print("Done.")

Saving best_model...
Done.


In [20]:
importances = best_model.get_feature_importance()
feature_names = best_model.feature_names_

feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
})

feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

print("--- Feature Importance Analysis ---")
print(f"Total features: {len(feature_importance_df)}")

print("\nTop 10 Most Important Features:")
print(feature_importance_df.head(10))

print("\nBottom 10 Least Important Features:")
print(feature_importance_df.tail(10))

n_features_to_remove = 100
n_features_to_keep = len(feature_importance_df) - n_features_to_remove
top_features = feature_importance_df.head(n_features_to_keep)['feature'].tolist()

print(f"\nKeeping top {len(top_features)} features and removing bottom {n_features_to_remove}.")

X_train_top_features = X_train_proc[top_features]
X_test_top_features = X_test_proc.reindex(columns=top_features, fill_value=0) 

original_cat_features = set(CAT_FEATURES)
top_features_set = set(top_features)
new_cat_features = list(original_cat_features.intersection(top_features_set))

print(f"Original categorical features: {len(CAT_FEATURES)}")
print(f"Categorical features kept: {len(new_cat_features)}")

best_params_from_optuna = study.best_trial.params

final_params_new = best_params_from_optuna.copy()
final_params_new.update({
    'iterations': 2000, 
    'eval_metric': 'Logloss',
    'task_type': 'CPU',
    'early_stopping_rounds': 50
})

print("\nRetraining model with top features...")
new_model = CatBoostClassifier(**final_params_new)

new_model.fit(
    X_train_top_features, y_train,
    eval_set=(X_test_top_features, y_test),
    cat_features=new_cat_features,
    verbose=False
)

y_preds_new = new_model.predict(X_test_top_features)
new_f1 = f1_score(y_test, y_preds_new, pos_label=1)

print("\n--- Model Performance Comparison ---")
# 'final_f1' comes from cell 30 in your notebook
print(f"Original F1 score (all features): {final_f1:.4f}")
print(f"New F1 score (top {len(top_features)} features): {new_f1:.4f}")

print("\nNew Model Classification Report (Top Features):")
print(classification_report(y_test, y_preds_new, target_names=['Class 0.0', 'Class 1.0']))

--- Feature Importance Analysis ---
Total features: 161

Top 10 Most Important Features:
                        feature  importance
47              liab_x_multicar   13.979516
48         liab_x_highrisk_site    5.244503
152  recovery_feasibility_score    4.422504
67     is_multi_vehicle_unclear    4.329813
45               liab_x_witness    3.914721
61                liab_prct_log    2.977464
58            liab_prct_squared    2.962211
53          multicar_x_highrisk    2.834882
62                 liab_inverse    2.651149
13                    liab_prct    2.645595

Bottom 10 Least Important Features:
               feature  importance
34           is_monday         0.0
100      light_vehicle         0.0
99       heavy_vehicle         0.0
96      luxury_vehicle         0.0
95        young_novice         0.0
35           is_friday         0.0
36               is_q4         0.0
37          claim_hour         0.0
91      veteran_driver         0.0
107  very_high_mileage         0.0

Keep

In [21]:
# Output module, from model_citizens.ipynb
real_test = pd.read_csv("data/Testing_TriGuard.csv")

X_real_test_proc = pre.transform(real_test)
X_real_test_proc = X_real_test_proc.reindex(columns=X_train_proc.columns, fill_value=0)
real_pred_proba = best_model.predict_proba(X_real_test_proc)[:, 1]
real_pred_label = (real_pred_proba >= 0.5).astype(int)

prediction = pd.DataFrame({
    "claim_number": real_test["claim_number"],
    "subrogation": real_pred_label
})

print(prediction.head())

Transforming data in 'catboost' mode...
Applying learned statistical transforms (Z-scores)...
CatBoost mode: Skipping target encoding application.
CatBoost mode: Dropping unused object/datetime columns...
Dropping: ['witness_present_ind', 'claim_date']
Transform complete.
   claim_number  subrogation
0       3126034            0
1       7380142            1
2       4655051            0
3       6728725            1
4       9848460            1


In [22]:
prediction.to_csv("results/catboost_6044_prediction.csv", index=False)